# Cohort, Outcome, and Clinical Elements Selection

This notebook utilizes the beneficiary, provider, and claims data sources in the Integrated Data Repository (IDR) - a high-volume data warehouse integrating Parts A, B, C, D, and DME claims - to build a predictive algorithm to identify beneficiaries likely to have an inpatient hospitalization occur within approximately 3 months. 

This notebook completes the following steps: <br>
       &emsp; 1. Specify the relevant cohort, <br>
       &emsp; 2. Define the outcome to be predicted, <br>
       &emsp; 3. Select the clinical elements that will be used in the algorithm

## **Prerequisites**

This notebook leverages the following packages.

In [ ]:
import snowflake.connector

## **Access the Data Repository**

Connect to the IDRC through a snowflake connector. 

In [2]:
con = snowflake.connector.connect(
    user= 'abcd',
    password= '*****',
    account= 'cms-idr.privatelink',
)

## **Step 1: Define the Cohort**
<br> &ensp;Create a cohort of Medicare beneficiaries with Part A and B FFS enrollment. Only include Non-ESRD and beneficiaries over age 65 as of 12/31/2018.

### **1.1 Input Data**
First, we will pull beneficiaries enrollment information from the IDRC using snowflake SQL in the IDRC and save results to an S3 bucket. The following fields were used as inputs:

|**Type** | **Variable** | **Definition**  | **Values**  |
| --- | --- | --- | --- |
|Demographic | BENE_SK | Unique identifier for each beneficiary | string|
|Demographic | BENE_BIRTH_DT | Beneficiary Birth Date| format YYYY-MM-DD |
|Demographic | BENE_DEATH_DT | Beneficiary Death Date| format YYYY-MM-DD |
|Demographic | BENE_RTI_RACE_CD | Beneficiary race code (modified using RTI algorithm) |0: Unknown <br>1: Non-Hispanic White <br> 2: Black (or African-American) <br> 3: Other <br> 4: Asian/Pacific Islander <br> 5: Hispanic <br>6: Native / Indigenous |
|Demographic | BENE_SEX_CD | Beneficiary sex code |0: Nonbinary <br> 1: Male <br>2: Female |
|Enrollment | BENE_LIS_IND_CD | Low income status indicator  |N: Not low income <br>Y: Low income|
|Enrollment | BENE_DUAL_STUS_CD |  Dual status indicator   | 01, 02, 03, 04, 05, 06, 08, 09: Dual Eligible <br>00, 99, NA: Not Dual Eligible|
|Demographic| GEO_URBN_RRL_CD |  Rural/urban field based on the RUCC code  | Rural <br> Urban <br> Unknown|
|Enrollment | BENE_PTA_STUS_CD | Medicare Part A Enrollment indicator | Y: Part A Enrollment <br>N: No Part A Enrollment |
|Enrollment | BENE_PTB_STUS_CD | Medicare Part B Enrollment indicator | Y: Part B Enrollment <br>N: No Part B Enrollment |
|Enrollment | BENE_PTD_STUS_CD | Medicare Part D Enrollment indicator | Y: Part D Enrollment <br>N: No Part D Enrollment |
|Enrollment | BENE_MDCR_STUS_CD | Medicare Status Code | 11, 21, 31: ESRD <br>0, 10, 20: Non-ESRD |
|Clinical | CLM_DGNS_CD |All ICD-10 CM Diagnosis codes for 2018  | string |
|Clinical |CLM_LINE_HCPCS_CD | Procedure codes for 2018 | string |
|Clinical |HCPCS_BETOS_CD | Betos groupings HCPC procedure codes for 2018  | string |
|Clinical |CLM_LINE_NDC_CD | NDC codes for 2018 | string |
|Clinical|CLASS_NO  | USP drug classes for 2018  | string |

*In order to execute SnowSQL statements, we used a database cursor.*

In [ ]:
con.cursor().execute("""DROP TABLE IF EXISTS CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_ALL;""")
                                  
con.cursor().execute("""                           
CREATE TABLE CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_ALL AS 
SELECT DISTINCT 
BENE_SK,
MIN(BENE_BIRTH_DT) AS BENE_BIRTH_DT, 
MIN(BENE_DEATH_DT) AS BENE_DEATH_DT, 
MAX(BENE_RTI_RACE_CD) AS BENE_RTI_RACE_CD,
MIN(BENE_SEX_CD) AS BENE_SEX_CD,
MIN(GEO_SK) AS GEO_SK,
 --If bene had Low Income Subsidy during 2018 or 2019
CASE WHEN MAX(CASE WHEN BENE_LIS_IND_CD = 'Y' THEN 1 ELSE 0 END) =1 THEN 1 ELSE 0 END AS LIS_STATUS,
 --If bene had Dual Status during 2018 or 2019
CASE WHEN MAX(CASE WHEN BENE_DUAL_STUS_CD IN ('01','02','03','04','05','06','08') THEN 1 ELSE 0 END)=1 THEN 1 ELSE 0 END AS DUAL_STATUS,
--Min and Max dates of Part A status else NULL
MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) AS MIN_ADATE,
MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) AS MAX_ADATE,
--Number of months of Part A Enrollment in 2018
CASE WHEN MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <= '2018-01-01' AND
          MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >= '2018-12-31' THEN 12
     WHEN (MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2018-01-01' AND '2018-12-31') AND
          (MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2018-01-01' AND '2018-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) ,
            MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END)))+1
     WHEN MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <'2018-01-01' AND
          MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2018-01-02' AND '2018-12-31' 
          THEN (DATEDIFF(MONTH, '2018-01-01',MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END))+1)
     WHEN (MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2018-01-01' AND '2018-12-31') AND
          (MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >'2018-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END),'2018-12-31'))+1
     ELSE 0 END AS PARTA_18,
 --Number of months of Part A Enrollment in 2019
 CASE WHEN MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <= '2019-01-01' AND
          MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >= '2019-12-31' THEN 12
     WHEN (MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2019-01-01' AND '2019-12-31') AND
          (MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2019-01-01' AND '2019-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) ,
            MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END)))+1
     WHEN MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <'2019-01-01' AND
          MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2019-01-02' AND '2019-12-31' 
          THEN (DATEDIFF(MONTH, '2019-01-01',MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END))+1)
     WHEN (MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2019-01-01' AND '2019-12-31') AND
          (MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >'2019-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END),'2019-12-31'))+1
     ELSE 0 END AS PARTA_19,
--Min and Max dates of Part B status else null
MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) AS MIN_BDATE,
MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) AS MAX_BDATE,
--Number of months of Part B Enrollment in 2018
CASE WHEN MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <= '2018-01-01' AND
          MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >= '2018-12-31' THEN 12
     WHEN (MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2018-01-01' AND '2018-12-31') AND
          (MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2018-01-01' AND '2018-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) ,
            MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END)))+1
     WHEN MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <'2018-01-01' AND
          MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2018-01-02' AND '2018-12-31' 
          THEN (DATEDIFF(MONTH, '2018-01-01',MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END))+1)
     WHEN (MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2018-01-01' AND '2018-12-31') AND
          (MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >'2018-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END),'2018-12-31'))+1
     ELSE 0 END AS PARTB_18,
 --Number of months of Part B Enrollment in 2019
 CASE WHEN MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <= '2019-01-01' AND
          MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >= '2019-12-31' THEN 12
     WHEN (MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2019-01-01' AND '2019-12-31') AND
          (MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2019-01-01' AND '2019-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) ,
            MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END)))+1
     WHEN MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <'2019-01-01' AND
          MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2019-01-02' AND '2019-12-31' 
          THEN (DATEDIFF(MONTH, '2019-01-01',MAX(CASE WHEN BENE_PTA_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END))+1)
     WHEN (MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2019-01-01' AND '2019-12-31') AND
          (MAX(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >'2019-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTB_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END),'2019-12-31'))+1
     ELSE 0 END AS PARTB_19,
--Min and Max dates of Part D status else null
MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) AS MIN_DDATE,
MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) AS MAX_DDATE,
--Number of months of Part D Enrollment in 2018
CASE WHEN MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <= '2018-01-01' AND
          MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >= '2018-12-31' THEN 12
     WHEN (MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2018-01-01' AND '2018-12-31') AND
          (MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2018-01-01' AND '2018-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) ,
            MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END)))+1
     WHEN MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <'2018-01-01' AND
          MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2018-01-02' AND '2018-12-31' 
          THEN (DATEDIFF(MONTH, '2018-01-01',MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END))+1)
     WHEN (MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2018-01-01' AND '2018-12-31') AND
          (MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >'2018-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END),'2018-12-31'))+1
     ELSE 0 END AS PARTD_18,
 --Number of months of Part D Enrollment in 2019
 CASE WHEN MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <= '2019-01-01' AND
          MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >= '2019-12-31' THEN 12
     WHEN (MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2019-01-01' AND '2019-12-31') AND
          (MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2019-01-01' AND '2019-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) ,
            MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END)))+1
     WHEN MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) <'2019-01-01' AND
          MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) BETWEEN '2019-01-02' AND '2019-12-31' 
          THEN (DATEDIFF(MONTH, '2019-01-01',MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END))+1)
     WHEN (MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END) BETWEEN '2019-01-01' AND '2019-12-31') AND
          (MAX(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_OBSLT_DT ELSE NULL END) >'2019-12-31') 
          THEN (DATEDIFF(MONTH, MIN(CASE WHEN BENE_PTD_STUS_CD = 'Y' THEN BENE_FCT_EFCTV_DT ELSE NULL END),'2019-12-31'))+1
     ELSE 0 END AS PARTD_19
FROM CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_BENE_FCT_TRANS_HSTRY 
WHERE  
	  BENE_FCT_EFCTV_DT <= '2019-12-31' AND
      BENE_FCT_OBSLT_DT >= '2018-01-01' AND /*Between Jan 1 2018 and Dec 31 2019*/
	  CNTRCT_PTC_NUM = 'UNK' AND /*ONLY FFS*/
	  BENE_BIRTH_DT < '1953-01-01' AND /*OVER 65 AS OF 2018*/	   
      BENE_MDCR_STUS_CD not in ('11','21','31') /*NON-ESRD*/ AND
	  IDR_TRANS_OBSLT_TS = '9999-12-31'
GROUP BY BENE_SK;
""")

Limit Cohort to only beneficiaries with at least one month of Part A and Part B FFS Enrollment in both 2018 and 2019

In [ ]:
con.cursor().execute("""DROP TABLE IF EXISTS CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A;""")
                                  
con.cursor().execute(""" CREATE TABLE CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A AS 
SELECT DISTINCT
BENE_SK,
BENE_BIRTH_DT,
BENE_DEATH_DT,
BENE_RACE_CD,
BENE_RTI_RACE_CD,
BENE_SEX_CD,
LIS_STATUS,
DUAL_STATUS,
GEO_FIPS_STATE_CD,
GEO_FIPS_CNTY_CD,
CASE WHEN GEO_URBN_RRL_CD IS NULL THEN 'UNKNOWN' ELSE GEO_URBN_RRL_CD END AS GEO_URBN_RRL_CD,
PARTA_18,
PARTB_18,
PARTD_18,
PARTA_19,
PARTB_19,
PARTD_19
FROM CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_ALL A LEFT JOIN CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_GEO_ZIP5_CD B ON A.GEO_SK=B.GEO_SK
 		AND GEO_ZIP5_ACTV_SW = 'Y'  
WHERE PARTA_18 >0 AND PARTA_19 >0 AND PARTB_18 >0 AND PARTB_19 >0; """)

## **Step 2: Define the Outcome Variable**
<br> The outcome variable, UNPLANNED_ADMISSION, is a binary variable indicating whether a Beneficiary had an inpatient hospitalization in the first 3 months of 2019. The definition of an "unplanned" admission originates from CMS' Planned Readmission Algorithm (V 4.0).

"The planned readmission algorithm uses a flowchart and four tables of specific AHRQ CCS procedure categories, AHRQ CCS diagnosis categories, and singular ICD-10 codes to classify readmissions as planned. Readmissions are considered planned if ANY of the following occurs during the readmission:

1) A procedure is performed that is in one of the procedure categories that are always planned regardless of diagnosis.

2) The principal diagnosis is in one of the diagnosis categories that are always planned. 3) A procedure is performed that is one of the defined potentially planned procedures and the principal diagnosis is not in the list of defined acute discharge diagnoses"

<ins>https://www.ahrq.gov/topics/hospital-readmissions.html<ins>

In [ ]:
con.cursor().execute("""DROP TABLE IF EXISTS CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A_OUTCOME_1;""")
con.cursor().execute("""DROP TABLE IF EXISTS CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A_OUTCOME;""")
                                  
con.cursor().execute("""CREATE TABLE CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A_OUTCOME_1 AS
select  
distinct
  bene_sk,
  // creates a flag for an unplanned admission 
  case when max(case when concat(pr1_ind,pr2_ind,pr3_ind,pr4_ind,diagnosis_ind) in ('00001','00111') then 1 else 0 end) =1 then 1 else 0
  end as unplanned_admission
from 
(
select distinct 
  t1.bene_sk,
  t1.clm_cntl_num,
  // admission is NOT for bone marrow, kidney, or any other organ transplant (table PR1 in supp file)
  case
  when clm_prcdr_1_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_2_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_3_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_4_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_5_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_7_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_8_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_9_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_10_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_11_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_12_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_13_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_14_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_15_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_16_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_17_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_18_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_19_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_20_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_21_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_22_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_23_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_24_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  when clm_prcdr_25_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR1) then '1'
  else '0' end as pr1_ind,
  // admission is NOT for maintenance chemotherapy or rehab (table PR2 in supp file)
  case
  when clm_prncpl_dgns_cd  in (select ICD_10_DIAG_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR2)then '1'
  else '0' end as pr2_ind,
  // admission is NOT for a potentially planned procedure (table PR3 in supp file) 
  case
  when  clm_prcdr_1_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when clm_prcdr_2_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when clm_prcdr_3_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when clm_prcdr_4_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when clm_prcdr_5_cd  in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_6_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_7_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_8_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_9_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_10_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_11_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_12_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_13_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_14_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_15_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_16_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_17_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_18_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_19_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_20_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_21_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_22_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_23_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_24_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  when  clm_prcdr_25_cd in  (select ICD_10_PROD_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR3) then '1'
  else '0' end as pr3_ind,
  //principal diagnosis code is acute or complications of care (table PR4 in supp file)
  case
  when clm_prncpl_dgns_cd  in (select ICD_10_DIAG_CD from IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.PR4)then '1'
  else '0' end as pr4_ind,
  // any instance of a procedure or diagnosis cd
 // case when  clm_prcdr_1_cd is not null then '1' else '0' end as procedure_ind,
  case when clm_prncpl_dgns_cd is not null then '1' else '0' end as diagnosis_ind
from IDRC_PRD.CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_CLM as t1
inner join IDRC_PRD.CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_CLM_INSTNL as t2
  on t1.GEO_BENE_SK=t2.GEO_BENE_SK and 
  t1.CLM_DT_SGNTR_SK=t2.CLM_DT_SGNTR_SK and 
  t1.CLM_NUM_SK=t2.CLM_NUM_SK and 
  t1.CLM_TYPE_CD=t2.CLM_TYPE_CD
inner join IDRC_PRD.CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_CLM_PROD_MTRLZD as t4
on t1.GEO_BENE_SK=t4.GEO_BENE_SK and 
  t1.CLM_DT_SGNTR_SK=t4.CLM_DT_SGNTR_SK and
  t1.CLM_NUM_SK=t4.CLM_NUM_SK and 
  t1.CLM_TYPE_CD=t4.CLM_TYPE_CD
where t1.clm_ffs_ind = 'Y' and 
  t1.clm_type_cd = 60 and 
  // retrict to enrollment through 2019
  t1.clm_from_dt between DATE'2019-01-01' and DATE'2019-12-31' and
  t1.clm_finl_actn_ind='Y' and 
  t1.clm_ltst_clm_ind='Y' and 
  t1.clm_pmt_amt > 0 and 
  t2.CLM_BILL_FAC_TYPE_CD = '1' and
  t2.CLM_BILL_CLSFCTN_CD IN ('1','2') and
  // selection code (replace with any defined cohort of benes)
  t1.bene_sk in (select distinct bene_sk from  IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A)
) 
group by bene_sk;""")
                     
con.cursor().execute("""CREATE TABLE CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A_OUTCOME AS 
select a.*, coalesce(unplanned_admission,0) as unplanned_admission
from CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A A LEFT JOIN CMS_ADM_PTDPYMT_MOD_PRD.AI_OUTCOME B ON A.BENE_SK=B.BENE_SK;""")

Copy the IDR table from the IDR Analytic Data Mart (ADM) to the Amazon Web Services (AWS) S3 Buckets

### **Output Table** 
<br> The resulting AI_COHORT_A_OUTCOME table uploaded to the S3 buckets include the following fields:

|**Type** | **Variable** | **Definition**  | **Values**  |
| --- | --- | --- | --- |
|Demographic | BENE_SK | Unique identifier for each beneficiary | string|
|Demographic | BENE_BIRTH_DT | Beneficiary Birth Date| format YYYY-MM-DD |
|Demographic | BENE_DEATH_DT | Beneficiary Death Date| format YYYY-MM-DD |
|Demographic | BENE_RTI_RACE_CD | Beneficiary race code (modified using RTI algorithm) |0: Unknown <br>1: Non-Hispanic White <br> 2: Black (or African-American) <br> 3: Other <br> 4: Asian/Pacific Islander <br> 5: Hispanic <br>6: Native / Indigenous |
|Demographic | BENE_SEX_CD | Beneficiary sex code |0: Nonbinary <br> 1: Male <br>2: Female |
|Enrollment | LIS_STATUS | Low income status indicator  |0: Not low income <br>1: Low income|
|Enrollment | DUAL_STATUS |  Dual status indicator   | 1: Dual Eligible <br>0: Not Dual Eligible|
|Demographic| GEO_URBN_RRL_CD |  Rural/urban field based on the RUCC code  | Rural <br> Urban <br> Unknown|
|Enrollment| PARTA_18 | 2018 Part A enrollment indicator / Number of months of Part A enrollment | 0-12  |
|Enrollment| PARTA_19 | 2019 Part A enrollment indicator / Number of months of Part A enrollment | 0-12  |
|Enrollment| PARTB_18 | 2018 Part B enrollment indicator / Number of months of Part B enrollment | 0-12  |
|Enrollment| PARTB_19 | 2019 Part B enrollment indicator / Number of months of Part B enrollment | 0-12  |
|Enrollment| PARTD_18 | 2018 Part D enrollment indicator / Number of months of Part D enrollment | 0-12  |
|Enrollment| PARTD_19 | 2019 Part D enrollment indicator / Number of months of Part D enrollment | 0-12  |
|Dependent Variable | UNPLANNED_ADMISSION | Outcome variable: indicator if the beneficiary had an unplanned inpatient hospitalization in the first 3 months of 2019 |  0: NO unplanned admission <br>1: Unplanned admission |

In [ ]:
con.cursor().execute("""copy into @IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.CMMI_AI_POC_STG/COHORT_A_OUTCOME from (select * from CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A_OUTCOME)
FILE_FORMAT =(TYPE=CSV FIELD_DELIMITER=',' COMPRESSION=gzip) SINGLE=TRUE HEADER=TRUE MAX_FILE_SIZE=4900000000;""")

## **Step 3: Select the clinical elements**

Pull all diagnosis codes from all claim types in 2018 for the selected Cohort. Put the Diagnosis codes and Diagnosis groups into arrays to condense the table.

### **Output Table**

The COHORT_DX_ARRAY table contains the following fields: 

|**Type** | **Variable** | **Definition**  | **Values**  |
| --- | --- | --- | --- |
|Demographic | BENE_SK | Unique identifier for each beneficiary | string|
|Array of Diagnosis Codes | DX_ARRAY |All ICD-10 CM Diagnosis codes for 2018  | string |
|Array of ICD-10 CM Diagnosis Groupings | DX_SUBGRP_ARRAY | A00-B99:  Certain infectious and parasitic diseases <br>C00-D49:  Neoplasms <br>D50-D89:  Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism <br>E00-E89:  Endocrine, nutritional and metabolic diseases <br>F01-F99:  Mental, Behavioral and Neurodevelopmental disorders <br>G00-G99:  Diseases of the nervous system <br>H00-H59:  Diseases of the eye and adnexa <br>H60-H95:  Diseases of the ear and mastoid process <br>I00-I99: Diseases of the circulatory system <br>J00-J99:  Diseases of the respiratory system <br>K00-K95:  Diseases of the digestive system <br>L00-L99:  Diseases of the skin and subcutaneous tissue <br>M00-M99:  Diseases of the musculoskeletal system and connective tissue <br>N00-N99:  Diseases of the genitourinary system <br>O00-O9A: Pregnancy, childbirth and the puerperium <br>P00-P96:  Certain conditions originating in the perinatal period <br>Q00-Q99:  Congenital malformations, deformations and chromosomal abnormalities <br>R00-R99:  Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified <br>S00-T88:  Injury, poisoning and certain other consequences of external causes <br>U00-U85:  Codes for special purposes <br>V00-Y99:  External causes of morbidity <br>Z00-Z99:  Factors influencing health status and contact with health services | string |



In [ ]:
con.cursor().execute("""DROP TABLE IF EXISTS CMS_ADM_PTDPYMT_MOD_PRD.COHORT_DX_ARRAY;""")

con.cursor().execute("""CREATE TABLE CMS_ADM_PTDPYMT_MOD_PRD.COHORT_DX AS
SELECT DISTINCT A.BENE_SK, 
CLM_DGNS_CD,
 CASE    WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'A00' AND 'B99' THEN '1_CERTAIN_INFECTIOUS_PARASITIC'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'C00' AND 'D49' THEN '2_NEOPLASMS'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'D50' AND 'D89' THEN '3_BLOOD'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'E00' AND 'E89' THEN '4_ENDOCRINE_NUTRITIONAL_METABOLIC'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'F01' AND 'F99' THEN '5_MENTAL_BEHAVIORAL'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'G00' AND 'G99' THEN '6_NERVOUS_DISEASES'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'H00' AND 'H59' THEN '7_EYE_ADNEXA_DISEASES'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'H60' AND 'H95' THEN '8_EAR_MASTOID_DISEASES'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'I00' AND 'I99' THEN '9_CIRCULATORY_DISEASES'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'J00' AND 'J99' THEN '10_RESPIRATORY_DISEASES'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'K00' AND 'K95' THEN '11_DIGESTIVE_DISEASES'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'L00' AND 'L99' THEN '12_SKIN_DISEASES'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'M00' AND 'M99' THEN '13_MUSCULOSKELETAL_DISEASES'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'N00' AND 'N99' THEN '14_GENITOURINARY_DISEASES'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'O00' AND 'O9A' THEN '15_PREGNANCY_CHILDBIRTH'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'P00' AND 'P96' THEN '16_PERINATAL'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'Q00' AND 'Q99' THEN '17_CONGENITAL_MALFORMATIONS'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'R00' AND 'R99' THEN '18_ABNORMAL_CLINICAL_FINDINGS'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'S00' AND 'T88' THEN '19_INJURY_POISIONING'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'U00' AND 'U85' THEN '20_SPECIAL_PURPOSES'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'V00' AND 'Y99' THEN '21_EXTERNAL_CAUSES_MORBIDITY'
         WHEN SUBSTR(CLM_DGNS_CD,1,3) BETWEEN 'Z00' AND 'Z99' THEN '22_FACTORS_INFLUENCING'
         ELSE NULL END AS DX_SUBGRP
FROM CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A A INNER JOIN
CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_CLM B ON A.BENE_SK=B.BENE_SK INNER JOIN
CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_CLM_PROD C ON B.GEO_BENE_SK=C.GEO_BENE_SK  AND 
	B.CLM_DT_SGNTR_SK=C.CLM_DT_SGNTR_SK AND B.CLM_TYPE_CD=C.CLM_TYPE_CD AND
	B.CLM_NUM_SK=C.CLM_NUM_SK
WHERE YEAR(CLM_THRU_DT) = 2018 AND /*Limit to 2018*/
B.CLM_TYPE_CD IN (10,20,30,40,50,60,71,72,81,82) AND /*All claim types*/ 
CLM_DGNS_CD IS NOT NULL AND 
CLM_FFS_IND = 'Y' AND /*Only FFS claims*/
CLM_FINL_ACTN_IND = 'Y';""")


con.cursor().execute("""CREATE TABLE CMS_ADM_PTDPYMT_MOD_PRD.COHORT_DX_ARRAY AS
SELECT DISTINCT BENE_SK, 
ARRAY_AGG(DISTINCT CLM_DGNS_CD) WITHIN GROUP (ORDER BY CLM_DGNS_CD) AS DX_ARRAY,
ARRAY_AGG(DISTINCT DX_SUBGRP) WITHIN GROUP (ORDER BY DX_SUBGRP)  AS DX_SUBGRP_ARRAY
FROM CMS_ADM_PTDPYMT_MOD_PRD.COHORT_DX_TEST
GROUP BY BENE_SK;""")
                     
con.cursor().execute("""copy into @IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.CMMI_AI_POC_STG/COHORT_A_DX_ZIP from (select * from CMS_ADM_PTDPYMT_MOD_PRD.COHORT_DX_ARRAY)
FILE_FORMAT =(TYPE=CSV FIELD_DELIMITER='\t' COMPRESSION=gzip) SINGLE=TRUE HEADER=TRUE MAX_FILE_SIZE=4900000000;""")					 

Pull all procedure codes and corresponding BETOS codes for all 2018 claims for the selected Cohort. Put the Procedure codes and BETOS codes into arrays to condense the table.

### **Output Table**

The COHORT_PROC_ARRAY table contains the following fields: 

|**Type** | **Variable** | **Definition**  | **Values**  |
| --- | --- | --- | --- |
|Demographic | BENE_SK | Unique identifier for each beneficiary | string|
|Array of Procedure Codes |PROC_ARRAY | Procedure codes for 2018 | string |
|Array of BETOS Codes|BETOS_ARRAY | Betos groupings HCPC procedure codes for 2018  | string |

In [ ]:
con.cursor().execute("""DROP TABLE IF EXISTS CMS_ADM_PTDPYMT_MOD_PRD.COHORT_PROC_ARRAY;""")

con.cursor().execute("""CREATE TABLE CMS_ADM_PTDPYMT_MOD_PRD.COHORT_PROC_ARRAY AS
SELECT DISTINCT A.BENE_SK, 
ARRAY_AGG(DISTINCT CLM_LINE_HCPCS_CD) WITHIN GROUP (ORDER BY CLM_LINE_HCPCS_CD) AS PROC_ARRAY,
ARRAY_AGG(DISTINCT HCPCS_BETOS_CD) WITHIN GROUP (ORDER BY HCPCS_BETOS_CD) AS BETOS_ARRAY
FROM CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A A INNER JOIN
CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_CLM B ON A.BENE_SK=B.BENE_SK INNER JOIN
CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_CLM_LINE C ON B.GEO_BENE_SK=C.GEO_BENE_SK  AND 
	B.CLM_DT_SGNTR_SK=C.CLM_DT_SGNTR_SK AND B.CLM_TYPE_CD=C.CLM_TYPE_CD AND
	B.CLM_NUM_SK=C.CLM_NUM_SK LEFT JOIN
CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_HCPCS_CD D ON C.CLM_LINE_HCPCS_CD=D.HCPCS_CD
	AND CLNDR_HCPCS_YR_NUM = 2018
WHERE YEAR(CLM_THRU_DT) = 2018 AND 
B.CLM_TYPE_CD IN (40,50,71,72,81,82) AND 
CLM_LINE_HCPCS_CD IS NOT NULL AND 
CLM_LINE_HCPCS_CD <> '' AND
CLM_FFS_IND = 'Y' AND 
CLM_FINL_ACTN_IND = 'Y'
GROUP BY A.BENE_SK;""")
                     
con.cursor().execute("""copy into @IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.CMMI_AI_POC_STG/COHORT_A_PROC_ZIP from (select * from CMS_ADM_PTDPYMT_MOD_PRD.COHORT_PROC_ARRAY)
FILE_FORMAT =(TYPE=CSV FIELD_DELIMITER='\t' COMPRESSION=gzip) SINGLE=TRUE HEADER=TRUE MAX_FILE_SIZE=4900000000;""")	

Pull all 2018 NDC and corresponding USP drug class for selected Cohort. Put the NDC and the RX Class numbers into arrays to condense the table.

### **Output Table**

The COHORT_RX_ARRAY table contains the following fields: 

|**Type** | **Variable** | **Definition**  | **Values**  |
| --- | --- | --- | --- |
|Demographic | BENE_SK | Unique identifier for each beneficiary | string|
|Array of NDCs |NDC_ARRAY | NDC codes for 2018 | string |
|Array of RX Classes|CLASS_ARRAY | USP drug classes for 2018  | string |

In [ ]:
con.cursor().execute("""DROP TABLE IF EXISTS CMS_ADM_PTDPYMT_MOD_PRD.COHORT_RX_ARRAY;""")

con.cursor().execute("""CREATE TABLE CMS_ADM_PTDPYMT_MOD_PRD.COHORT_RX_ARRAY AS
SELECT DISTINCT A.BENE_SK, 
ARRAY_AGG(DISTINCT CLM_LINE_NDC_CD) WITHIN GROUP (ORDER BY CLM_LINE_NDC_CD) AS NDC_ARRAY,
ARRAY_AGG(DISTINCT TO_VARCHAR(CLASS_NO)) WITHIN GROUP (ORDER BY TO_VARCHAR(CLASS_NO)) AS CLASS_ARRAY
FROM CMS_ADM_PTDPYMT_MOD_PRD.AI_COHORT_A A INNER JOIN
CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_CLM B ON A.BENE_SK=B.BENE_SK INNER JOIN
CMS_VDM_VIEW_MDCR_PRD.V2_MDCR_CLM_LINE C ON B.GEO_BENE_SK=C.GEO_BENE_SK  AND 
	B.CLM_DT_SGNTR_SK=C.CLM_DT_SGNTR_SK AND B.CLM_TYPE_CD=C.CLM_TYPE_CD AND
	B.CLM_NUM_SK=C.CLM_NUM_SK LEFT JOIN
CMS_ADM_PTDPYMT_MOD_PRD.CSZBRXNORM_USPDC_202209 D ON C.CLM_LINE_NDC_CD=D.NDC
WHERE YEAR(CLM_THRU_DT) = 2018 AND 
B.CLM_TYPE_CD IN (1,2,4) AND 
CLM_LINE_NDC_CD IS NOT NULL AND 
CLM_LINE_NDC_CD <> '' AND
CLM_FINL_ACTN_IND = 'Y'
GROUP BY A.BENE_SK;""")
                     
con.cursor().execute("""copy into @IDRC_PRD.CMS_ADM_PTDPYMT_MOD_PRD.CMMI_AI_POC_STG/COHORT_A_RX_ZIP from (select * from CMS_ADM_PTDPYMT_MOD_PRD.COHORT_RX_ARRAY)
FILE_FORMAT =(TYPE=CSV FIELD_DELIMITER='\t' COMPRESSION=gzip) SINGLE=TRUE HEADER=TRUE MAX_FILE_SIZE=4900000000;""")